In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim
import torch.autograd as autograd
import torchtext.vocab as torchvocab
from torch.autograd import Variable
import tqdm
import os
import time
import re
import pandas as pd
import string
import gensim
import time
import snowballstemmer
import collections
from collections import Counter
from nltk.corpus import stopwords
from itertools import chain
from sklearn.metrics import accuracy_score

In [2]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    # Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    stemmer = snowballstemmer.stemmer('english')
    stemmed_words = [stemmer.stemWord(word) for word in text]
    text = " ".join(stemmed_words)
    return text

In [3]:
# def readIMDB(path, seg='train'):
#     pos_or_neg = ['pos', 'neg']
#     data = []
#     for label in pos_or_neg:
#         files = os.listdir(os.path.join(path, seg, label))
#         for file in files:
#             with open(os.path.join(path, seg, label, file), 'r', encoding='utf8') as rf:
#                 review = rf.read().replace('\n', '')
#                 if label == 'pos':
#                     data.append([clean_text(review), 1])
#                 elif label == 'neg':
#                     data.append([clean_text(review), 0])
#     return data

def readIMDB(path, seg='train'):
    pos_or_neg = ['pos', 'neg']
    data = []
    for label in pos_or_neg:
        files = os.listdir(os.path.join(path, seg, label))
        for file in files:
            with open(os.path.join(path, seg, label, file), 'r', encoding='utf8') as rf:
                review = rf.read().replace('\n', '')
                if label == 'pos':
                    data.append([review, 1])
                elif label == 'neg':
                    data.append([review, 0])
    return data

In [4]:
train_data = readIMDB('aclImdb')
test_data = readIMDB('aclImdb', 'test')

In [5]:
def tokenizer(text):
    return [tok.lower() for tok in text.split(' ')]

train_tokenized = []
test_tokenized = []
for review, score in train_data:
    train_tokenized.append(tokenizer(review))
for review, score in test_data:
    test_tokenized.append(tokenizer(review))

In [6]:
vocab = set(chain(*train_tokenized))
vocab_size = len(vocab)

In [7]:
wvmodel = gensim.models.KeyedVectors.load_word2vec_format('test_word.txt',
                                                          binary=False, encoding='utf-8')

In [8]:
word_to_idx = {word: i+1 for i, word in enumerate(vocab)}
word_to_idx['<unk>'] = 0
idx_to_word = {i+1: word for i, word in enumerate(vocab)}
idx_to_word[0] = '<unk>'

In [9]:
def encode_samples(tokenized_samples, vocab):
    features = []
    for sample in tokenized_samples:
        feature = []
        for token in sample:
            if token in word_to_idx:
                feature.append(word_to_idx[token])
            else:
                feature.append(0)
        features.append(feature)
    return features

In [10]:
def pad_samples(features, maxlen=500, PAD=0):
    padded_features = []
    for feature in features:
        if len(feature) >= maxlen:
            padded_feature = feature[:maxlen]
        else:
            padded_feature = feature
            while(len(padded_feature) < maxlen):
                padded_feature.append(PAD)
        padded_features.append(padded_feature)
    return padded_features

In [11]:
train_features = torch.tensor(pad_samples(encode_samples(train_tokenized, vocab)))
train_labels = torch.tensor([score for _, score in train_data])
test_features = torch.tensor(pad_samples(encode_samples(test_tokenized, vocab)))
test_labels = torch.tensor([score for _, score in test_data])

In [12]:
class SentimentNet(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 bidirectional, weight, labels, use_gpu, **kwargs):
        super(SentimentNet, self).__init__(**kwargs)
        self.num_hiddens = num_hiddens
        self.use_gpu = use_gpu
        self.batch_size = batch_size
        self.bidirectional = bidirectional
        self.embedding = nn.Embedding.from_pretrained(weight)
#         self.embedding = nn.Embedding(vocab_size, embed_size)
#         self.embedding.weight.data[1:(len(weight)+1), :] = weight
        self.embedding.weight.requires_grad = False
        self.encoder = nn.LSTM(input_size=embed_size, hidden_size=self.num_hiddens,
                               num_layers=num_layers, bidirectional=self.bidirectional,
                               dropout=0)
#         if self.bidirectional:
#             self.decoder = nn.Linear(num_hiddens * 2, labels)
#         else:
#             self.decoder = nn.Linear(num_hiddens * 1, labels)
        self.decoder = nn.Linear(num_hiddens * 2, out_features=2)
#         self.softmax = nn.LogSoftmax()

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        states, hidden = self.encoder(
            embeddings.view(inputs.shape[1], len(inputs), -1))
        encoding = torch.cat([hidden[0][0], hidden[-1][0]], dim=1)
        outputs = self.decoder(encoding)
#         encoding = F.dropout(encoding, training=self.training)
#         outputs = self.decoder(hidden[-1][0])
#         outputs = F.log_softmax(outputs)
        return outputs

In [13]:
num_epochs = 5
embed_size = 100
num_hiddens = 100
num_layers = 2
bidirectional = True
batch_size = 64
labels = 2
device = torch.device('cuda:0')
use_gpu = True

In [14]:
weight = nn.Embedding(vocab_size+1, embed_size).weight.data

In [15]:
for i in range(len(wvmodel.index2word)):
    try:
        index = word_to_idx[wvmodel.index2word[i]]
    except:
        continue
    weight[index, :] = torch.tensor(wvmodel.get_vector(idx_to_word[word_to_idx[wvmodel.index2word[i]]]))

In [16]:
net = SentimentNet(vocab_size=(vocab_size+1), embed_size=embed_size,
                   num_hiddens=num_hiddens, num_layers=num_layers,
                   bidirectional=bidirectional, weight=weight,
                   labels=labels, use_gpu=True)
net.to(device)
loss_function = nn.CrossEntropyLoss()
# loss_function = nn.NLLLoss()
optimizer = optim.SGD(net.parameters(), lr=0.8)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.1)

In [17]:
train_set = torch.utils.data.TensorDataset(train_features, train_labels)
test_set = torch.utils.data.TensorDataset(test_features, test_labels)

In [18]:
train_iter = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                         shuffle=True, num_workers=2)
test_iter = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                        shuffle=False, num_workers=2)

In [ ]:
for epoch in range(num_epochs):
    start = time.time()
    train_loss, test_loss = 0, 0
    train_acc, test_acc = 0, 0
    n, m = 0, 0
    for feature, label in train_iter:
        n += 1
#         net.train()
        net.zero_grad()
        feature = Variable(feature.cuda())
        label = Variable(label.cuda())
        score = net(feature)
        loss = loss_function(score, label)
        loss.backward()
#         scheduler.step()
        optimizer.step()
        train_acc += accuracy_score(torch.argmax(score.cpu().data,
                                                 dim=1), label.cpu())
        train_loss += loss
    with torch.no_grad():
        for feature, label in test_iter:
            m += 1
#             net.eval()
            feature = feature.cuda()
            label = label.cuda()
            score = net(feature)
            loss = loss_function(score, label)
            test_acc += accuracy_score(torch.argmax(score.cpu().data,
                                                    dim=1), label.cpu())
            test_loss += loss
    end = time.time()
    runtime = end - start
    print('epoch: %d, train loss: %.4f, train acc: %.2f, test loss: %.4f, test acc: %.2f, time: %.2f' %
          (epoch, train_loss.data / n, train_acc / n, test_loss.data / m, test_acc / m, runtime))
#     print('epoch: %d, loss: %.4f, lr: %e'
#           % (epoch, losses.data / n, scheduler.get_lr()[0]))

epoch: 0, train loss: 10.6311, train acc: 0.50, test loss: 29.6891, test acc: 0.50, time: 52.53
